# Module 05 - Notebook 04: Similarity Search

## Learning Objectives
- Master cosine similarity and distance metrics
- Implement k-NN search algorithms
- Optimize search performance
- Handle edge cases and thresholds
- Build hybrid search (semantic + keyword)

---

## 1. Distance Metrics

Different ways to measure similarity between vectors:

### Cosine Similarity
- Measures angle between vectors
- Range: [-1, 1] (1 = identical, -1 = opposite)
- **Most common** for text embeddings

### Euclidean Distance
- Straight-line distance
- Range: [0, ∞] (0 = identical)
- Sensitive to magnitude

### Dot Product
- Sum of element-wise products
- Fast to compute
- Not normalized

## 2. Setup

In [ ]:
!pip install -q numpy scikit-learn sentence-transformers chromadb openai python-dotenv

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sentence_transformers import SentenceTransformer
import chromadb
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
model = SentenceTransformer('all-MiniLM-L6-v2')
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✓ Ready")

## 3. Comparing Distance Metrics

In [ ]:
# Create sample embeddings
texts = [
    "I love machine learning",
    "I enjoy studying AI",
    "The weather is nice"
]

embeddings = model.encode(texts)

# Cosine similarity
cos_sim = cosine_similarity(embeddings)
print("Cosine Similarity Matrix:")
print(cos_sim)
print()

# Euclidean distance
euc_dist = euclidean_distances(embeddings)
print("Euclidean Distance Matrix:")
print(euc_dist)
print()

# Dot product
dot_prod = np.dot(embeddings, embeddings.T)
print("Dot Product Matrix:")
print(dot_prod)

## 4. k-Nearest Neighbors (k-NN) Search

In [ ]:
def knn_search(query_embedding, document_embeddings, k=3, metric='cosine'):
    """
    Find k most similar documents.
    
    Args:
        query_embedding: Query vector
        document_embeddings: Document vectors
        k: Number of results
        metric: 'cosine' or 'euclidean'
    
    Returns:
        Indices of top k documents and their scores
    """
    if metric == 'cosine':
        similarities = cosine_similarity([query_embedding], document_embeddings)[0]
        top_k_indices = np.argsort(similarities)[::-1][:k]
        scores = similarities[top_k_indices]
    else:  # euclidean
        distances = euclidean_distances([query_embedding], document_embeddings)[0]
        top_k_indices = np.argsort(distances)[:k]
        scores = distances[top_k_indices]
    
    return top_k_indices, scores

# Test k-NN search
documents = [
    "Python programming tutorial",
    "Machine learning with Python",
    "Cooking delicious pasta",
    "Basketball game highlights",
    "Deep learning neural networks"
]

doc_embeddings = model.encode(documents)
query = "I want to learn AI"
query_emb = model.encode(query)

indices, scores = knn_search(query_emb, doc_embeddings, k=3)

print(f"Query: '{query}'\n")
print("Top 3 Results:\n")
for i, (idx, score) in enumerate(zip(indices, scores), 1):
    print(f"{i}. [{score:.3f}] {documents[idx]}")

## 5. Threshold-Based Filtering

In [ ]:
def threshold_search(query_embedding, document_embeddings, documents, threshold=0.5):
    """
    Return only documents above similarity threshold.
    """
    similarities = cosine_similarity([query_embedding], document_embeddings)[0]
    
    # Filter by threshold
    above_threshold = similarities >= threshold
    filtered_indices = np.where(above_threshold)[0]
    
    # Sort by similarity
    sorted_indices = filtered_indices[np.argsort(similarities[filtered_indices])[::-1]]
    
    results = [
        {"document": documents[idx], "similarity": similarities[idx]}
        for idx in sorted_indices
    ]
    
    return results

# Test with different thresholds
for threshold in [0.3, 0.5, 0.7]:
    results = threshold_search(query_emb, doc_embeddings, documents, threshold)
    print(f"\nThreshold = {threshold}: Found {len(results)} documents")
    for r in results:
        print(f"  [{r['similarity']:.3f}] {r['document']}")

## 6. Approximate Nearest Neighbors (ANN)

For large datasets, exact search is slow. ANN trades accuracy for speed.

In [ ]:
import time

# Simulate large dataset
n_documents = 10000
embedding_dim = 384

# Generate random embeddings (simulating a large corpus)
large_corpus_embeddings = np.random.randn(n_documents, embedding_dim).astype('float32')

# Normalize (important for cosine similarity)
large_corpus_embeddings = large_corpus_embeddings / np.linalg.norm(
    large_corpus_embeddings, axis=1, keepdims=True
)

query_vec = np.random.randn(embedding_dim).astype('float32')
query_vec = query_vec / np.linalg.norm(query_vec)

# Exact search
start = time.time()
exact_similarities = np.dot(large_corpus_embeddings, query_vec)
top_k_exact = np.argsort(exact_similarities)[::-1][:5]
exact_time = time.time() - start

print(f"Exact search on {n_documents:,} vectors: {exact_time*1000:.2f}ms")
print(f"Top 5 indices: {top_k_exact}")
print(f"Top 5 scores: {exact_similarities[top_k_exact]}")

print("\n💡 For very large datasets, use specialized libraries:")
print("  • FAISS (Facebook)")
print("  • Annoy (Spotify)")
print("  • HNSW (Hierarchical Navigable Small World)")

## 7. Hybrid Search (Semantic + Keyword)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

class HybridSearch:
    """
    Combine semantic (embeddings) and keyword (TF-IDF) search.
    """
    
    def __init__(self, semantic_weight=0.7):
        self.semantic_weight = semantic_weight
        self.keyword_weight = 1 - semantic_weight
        self.tfidf = TfidfVectorizer()
        self.documents = []
        self.semantic_embeddings = None
        self.keyword_matrix = None
    
    def index(self, documents):
        """Index documents for both semantic and keyword search."""
        self.documents = documents
        
        # Semantic embeddings
        self.semantic_embeddings = model.encode(documents)
        
        # Keyword vectors (TF-IDF)
        self.keyword_matrix = self.tfidf.fit_transform(documents)
    
    def search(self, query, k=3):
        """Hybrid search combining both approaches."""
        # Semantic search
        query_embedding = model.encode(query)
        semantic_scores = cosine_similarity(
            [query_embedding],
            self.semantic_embeddings
        )[0]
        
        # Keyword search
        query_tfidf = self.tfidf.transform([query])
        keyword_scores = cosine_similarity(
            query_tfidf,
            self.keyword_matrix
        )[0]
        
        # Combine scores
        hybrid_scores = (
            self.semantic_weight * semantic_scores +
            self.keyword_weight * keyword_scores
        )
        
        # Get top k
        top_indices = np.argsort(hybrid_scores)[::-1][:k]
        
        return [
            {
                "document": self.documents[idx],
                "score": hybrid_scores[idx],
                "semantic": semantic_scores[idx],
                "keyword": keyword_scores[idx]
            }
            for idx in top_indices
        ]

# Test hybrid search
corpus = [
    "Python is a programming language",
    "Machine learning with Python",
    "Deep learning algorithms",
    "Python snake in the jungle",  # Keyword match but wrong semantic
    "AI and artificial intelligence"
]

hybrid = HybridSearch(semantic_weight=0.7)
hybrid.index(corpus)

test_query = "Python for AI"
results = hybrid.search(test_query, k=3)

print(f"Query: '{test_query}'\n")
print("Hybrid Search Results:\n")
for i, r in enumerate(results, 1):
    print(f"{i}. [score: {r['score']:.3f}] {r['document']}")
    print(f"   Semantic: {r['semantic']:.3f} | Keyword: {r['keyword']:.3f}\n")

## 8. Handling Edge Cases

In [ ]:
# Edge case 1: Empty query
try:
    empty_emb = model.encode("")
    print(f"Empty query embedding shape: {empty_emb.shape}")
except Exception as e:
    print(f"Empty query error: {e}")

# Edge case 2: Very long text
long_text = "word " * 1000
long_emb = model.encode(long_text)
print(f"\nLong text ({len(long_text)} chars) → {long_emb.shape}")

# Edge case 3: No results above threshold
unrelated_query = "quantum physics"
tech_docs = ["cooking recipe", "gardening tips", "travel guide"]
tech_embs = model.encode(tech_docs)
unrelated_emb = model.encode(unrelated_query)

results = threshold_search(unrelated_emb, tech_embs, tech_docs, threshold=0.7)
print(f"\nUnrelated query '{unrelated_query}': {len(results)} results above 0.7")

# Edge case 4: Identical documents
duplicate_docs = ["same text", "same text", "different text"]
dup_embs = model.encode(duplicate_docs)
dup_query_emb = model.encode("same text")

similarities = cosine_similarity([dup_query_emb], dup_embs)[0]
print(f"\nDuplicate handling:")
for doc, sim in zip(duplicate_docs, similarities):
    print(f"  [{sim:.3f}] {doc}")

## Exercise: Build an Advanced Search Engine

Create a search engine with multiple features.

In [ ]:
# TODO: Complete this exercise
class AdvancedSearchEngine:
    """
    Advanced search with multiple strategies.
    """
    
    def __init__(self):
        # TODO: Initialize
        pass
    
    def index_documents(self, documents, metadata=None):
        """Index documents with metadata."""
        # TODO: Implement
        pass
    
    def search(
        self,
        query: str,
        k: int = 5,
        threshold: float = None,
        filters: dict = None,
        hybrid: bool = False
    ):
        """
        Search with options:
        - k: Number of results
        - threshold: Minimum similarity
        - filters: Metadata filters
        - hybrid: Use hybrid search
        """
        # TODO: Implement
        pass

# Test your engine
# engine = AdvancedSearchEngine()
# engine.index_documents([...])
# results = engine.search("query", hybrid=True)
# print(results)

## Summary

You learned:
- ✅ Distance metrics (cosine, euclidean, dot product)
- ✅ k-NN search implementation
- ✅ Threshold-based filtering
- ✅ Hybrid search (semantic + keyword)
- ✅ Handling edge cases

## Best Practices
1. **Use cosine similarity** for text embeddings
2. **Set appropriate thresholds** to filter noise
3. **Combine semantic + keyword** for best results
4. **Handle edge cases** (empty, duplicates, no results)
5. **Use ANN libraries** for large datasets

## Next Steps
- 📘 Notebook 05: Chunking Strategies